In [12]:
import torch
from torch import nn
import random
from pprint import pprint

import numpy as np

In [13]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(str(device) + ' will be used')

cpu will be used


In [23]:
def __number_sampler(max_digit=3):
    digits = ['0','1','2','3','4','5','6','7','8','9']
    k = random.randint(1,max_digit)
    string = ''.join(random.sample(digits,k))
    if string[0] == '0':
        return __number_sampler(max_digit=max_digit)
    return string

def __operator_sampler():
    operators = ['+','-','/','*']
    return random.choice(operators)

def get_random_input(n_input=100, max_len_limit=20, max_digit=3):
    ret = []
    for i in range(n_input):
        length = random.randint(1,max_len_limit)
        length = length+1 if length%2==1 else length
        length = max_len_limit # added for simplicity, later delete this line to make problem more complicated
        s = ''
        for j in range(length+1):
            if j % 2 == 0:
                s += __number_sampler(max_digit=max_digit)
            else:
                s += __operator_sampler()
        ret.append(s)
    return ret

def __one_hot_encoding(alphabet, max_len):
    x = np.zeros((len(alphabet),len(alphabet)),dtype=np.int32)
    dict_ = {}
    for i in range(x.shape[0]):
        x[i,i] = 1
        dict_[alphabet[i]] = x[i]
    return dict_

def create_input_seq(alphabet, max_len, input_sequence):
    alphabet2encod = __one_hot_encoding(alphabet, max_len)
    ret = np.zeros((len(input_sequence),len(input_sequence[0]),len(alphabet)))
    for j, inp in enumerate(input_sequence):
        temp = np.zeros((len(inp),len(alphabet)))
        for i in range(temp.shape[0]):
            temp[i,:] = alphabet2encod[inp[i]]
        ret[j,:,:] = temp
    return ret

In [24]:
class RNNModel(nn.Module):
    def __init__(self, input_feature_num, hidden_feature_num=20, n_layers=1, out_dim=1):
        super(RNNModel, self).__init__()
        
        self.input_feature_num = input_feature_num
        self.hidden_feature_num = hidden_feature_num
        self.n_layers = n_layers
        self.out_dim = out_dim
        
        self.rnn = nn.RNN(input_feature_num, hidden_feature_num, n_layers)
        self.fc = nn.Linear(hidden_feature_num, out_dim)
        
    def forward(self, x):
        batch_size = x.size(1)
        hidden = torch.zeros(self.n_layers, batch_size, self.hidden_feature_num)
        out, hidden = self.rnn(x, hidden)
        hidden = hidden.squeeze(0)
        out = self.fc(hidden)
        return out

In [32]:
max_len = 4
n_input = 10000
alphabet = ['0','1','2','3','4','5','6','7','8','9','-','+','/','*'] 
target_values = []
input_feature_num = len(alphabet)

input_sequence = get_random_input(n_input=n_input, max_len_limit=max_len, max_digit=1)
one_hot_input = create_input_seq(alphabet, max_len, input_sequence)
input_seq = torch.FloatTensor(one_hot_input).permute(1,0,2)

for i in range(len(input_sequence)):
    target_values.append(eval(input_sequence[i]))

target_values = torch.FloatTensor(target_values).unsqueeze(1)

In [ ]:
print()

In [28]:
model = RNNModel(input_feature_num=input_feature_num)
model.to(device)

n_epochs = 100
lr=0.01

loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [29]:
for epoch in range(1,n_epochs+1):
    optimizer.zero_grad()
    input_seq.to(device)
    output = model(input_seq)
    loss = loss_fn(output, target_values)
    
    loss.backward()
    optimizer.step()
    
    if epoch%10 == 0:
        print('Epoch: {}/{}.............'.format(epoch, n_epochs), end=' ')
        print("Loss: {:.4f}".format(loss.item()))

Epoch: 10/100............. Loss: 2646323.7500
Epoch: 20/100............. Loss: 2646051.5000
Epoch: 30/100............. Loss: 2645785.7500
Epoch: 40/100............. Loss: 2645535.7500
Epoch: 50/100............. Loss: 2645302.7500
Epoch: 60/100............. Loss: 2645086.0000
Epoch: 70/100............. Loss: 2644884.7500
Epoch: 80/100............. Loss: 2644697.5000
Epoch: 90/100............. Loss: 2644523.2500
Epoch: 100/100............. Loss: 2644361.0000
